# Q1

In [3]:
!curl localhost:9200

{
  "name" : "521e12fcfbb9",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "_5i-gZWtSZGHl59kU12Chw",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


Answer: `42f05b9372a9a4a470db3b52817899b99a76ee73`

# Getting the data

In [4]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

# Q2

In [12]:
from elasticsearch import Elasticsearch
from tqdm import tqdm

In [9]:
es_client = Elasticsearch('http://localhost:9200')

In [11]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions-homework"

es_client.indices.create(index=index_name, body=index_settings)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'course-questions-homework'}

In [13]:
for doc in tqdm(documents):
    es_client.index(index=index_name, body=doc)

100%|██████████████████████████████████████████████████████| 948/948 [00:03<00:00, 243.53it/s]


Answer: `index`

# Q3

In [14]:
query = "How do I execute a command in a running docker container?"

In [26]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }
    res = es_client.search(index=index_name, body=search_query)

    return res

In [ ]:
res = elastic_search(query)
res['hits']['max_score']

Answer: 84.05

# Q4

In [29]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    res = es_client.search(index=index_name, body=search_query)

    return res

In [30]:
res = elastic_search(query)

In [37]:
res['hits']['hits'][2]['_source']['question']

'How do I copy files from a different folder into docker container’s working directory?'

Answer 👆

# Q5

In [63]:
def build_prompt(query, search_results):
    context_template = """   
Q: {question}
A: {text}
    """.strip()

    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
    """.strip()

    context = ""

    for res in search_results:
        doc = res['_source']
        context = context + context_template.format(question=doc['question'], text=doc['text']) + '\n\n'
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [64]:
search_results = res['hits']['hits']
prompt = build_prompt(query, search_results)

In [65]:
len(prompt)

1462

Answer: 1462

# Q6

In [67]:
!pip install tiktoken

  Using cached regex-2024.5.15-cp310-cp310-macosx_11_0_arm64.whl.metadata (40 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.8/906.8 kB 4.4 MB/s eta 0:00:00 MB/s eta 0:00:01:01
Using cached regex-2024.5.15-cp310-cp310-macosx_11_0_arm64.whl (278 kB)


In [68]:
import tiktoken

In [69]:
encoding = tiktoken.encoding_for_model('gpt-4o')

In [71]:
len(encoding.encode(prompt))

322

Answer: 322

# Q Bonus 1

In [75]:
import os
from openai import OpenAI

In [76]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [77]:
openai_client = OpenAI(api_key=OPENAI_API_KEY)

In [78]:
def llm(prompt, client):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [79]:
ans = llm(prompt, openai_client)

In [80]:
print(ans)

To execute a command in a running Docker container, you need to use the `docker exec` command. Here are the steps:

1. First, find the container ID of the running container by using the command:
   ```bash
   docker ps
   ```

2. Once you have the container ID, execute a command inside the container using:
   ```bash
   docker exec -it <container-id> <command>
   ```

For example, to start a bash session inside the container, you would use:
```bash
docker exec -it <container-id> bash
```


# Q Bonus 2

In [102]:
input_token_price = 5 / 1_000_000
output_token_price = 15 / 1_000_000

n_requests = 1000
avg_input = 150
avg_output = 250

total_cost = n_requests * (avg_input * input_token_price + avg_output * output_token_price)
print(f"${round(total_cost, 2)}")

$4.5
